In [2]:
import pandas as pd
import pandas_profiling
import os
import pickle
import gc
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold



#評価用

train = pd.read_csv("data_EDA/train.csv")
test = pd.read_csv("data_EDA/test.csv")

with open('ensamble/rf_train.pickle', mode='br') as fi:
    train_p1 = pickle.load(fi)
    
with open('ensamble/lgb_train.pickle', mode='br') as fi:
    train_p2 = pickle.load(fi)
    
with open('ensamble/tabnet_w_train.pickle', mode='br') as fi:
    train_p3 = pickle.load(fi)
    
with open('ensamble/nn_train.pickle', mode='br') as fi:
    train_p4 = pickle.load(fi)
    
with open('ensamble/lgr_train.pickle', mode='br') as fi:
    train_p5 = pickle.load(fi)






#データ読み込み
with open('ensamble/rf_test.pickle', mode='br') as fi:
    test_p1 = pickle.load(fi)
    
with open('ensamble/lgb_test.pickle', mode='br') as fi:
    test_p2 = pickle.load(fi)
    
with open('ensamble/tabnet_w_test.pickle', mode='br') as fi:
    test_p3 = pickle.load(fi)
    
with open('ensamble/nn_test.pickle', mode='br') as fi:
    test_p4 = pickle.load(fi)
    
with open('ensamble/lgr_test.pickle', mode='br') as fi:
    test_p5 = pickle.load(fi)
    

## アンサンブル

In [3]:
#　評価用
df_train = pd.concat(
    [train['Outcome'],train_p1['pred'],train_p2['pred'], train_p3['pred'], train_p4['pred'], train_p5['pred']], 
                     axis=1, ignore_index=True).rename(columns={
    0: 'true', 1: 'pred1', 2: 'pred2', 3:'pred3', 4:'pred4',5:'pred5'
})

df_train['ensemble'] = df_train['pred1']*0.4 + df_train['pred2']*0.3 +df_train['pred3']*0.3 #+ df_train['pred4']*0.1 +  df_train['pred5']*0.1

# 提出用
df_test = pd.concat(
    [test['index'],test_p1['pred'],test_p2['pred'], test_p3['pred'],  test_p4['pred'], test_p5['pred']], 
                    axis=1, ignore_index=True).rename(columns={
    0: 'index', 1: 'pred1', 2: 'pred2', 3:'pred3', 4:'pred4', 5:'pred5'
})

df_test['ensemble'] = df_test['pred1']*0.4 + df_test['pred2']*0.3  +df_test['pred3']*0.3 #+ df_test['pred4']*0.1 + df_test['pred5']*0.1


def evaluate_ensemble(input_df, col_pred):
    print('[auc] model1:{:.4f},model2:{:.4f},model3:{:.4f},model4:{:.4f},model5:{:.4f}  ->    emsamble]{:.4f}'.format(
        roc_auc_score(input_df['true'], input_df['pred1']),
        roc_auc_score(input_df['true'], input_df['pred2']),
        roc_auc_score(input_df['true'], input_df['pred3']),
        roc_auc_score(input_df['true'], input_df['pred4']),
        roc_auc_score(input_df['true'], input_df['pred5']),
        
        
        roc_auc_score(input_df['true'], input_df[col_pred]),
    ))

In [4]:
df_train.head(20)

,true,pred1,pred2,pred3,pred4,pred5,ensemble
0,1.0,0.463801,0.628059,0.663979,0.410944,0.402233,0.573132
1,0.0,0.083710,0.053503,0.080110,0.109748,0.073798,0.073568
2,0.0,0.497738,0.371698,0.792315,0.385019,0.412125,0.548299
3,0.0,0.638009,0.586979,0.982137,0.447461,0.602159,0.725938
4,1.0,0.800905,0.561965,0.993841,0.492107,0.689416,0.787104
5,0.0,0.061086,0.119255,0.287218,0.125439,0.173991,0.146376
6,1.0,0.090498,0.051957,0.186049,0.113324,0.124462,0.107601
7,1.0,0.518100,0.373684,0.889087,0.463715,0.519163,0.586071
8,0.0,0.185520,0.118094,0.284754,0.210352,0.202901,0.195063
9,0.0,0.352941,0.288413,0.856990,0.344568,0.347957,0.484797


In [5]:
evaluate_ensemble(df_train, col_pred='ensemble')

[auc] model1:0.8053,model2:0.8182,model3:0.7669,model4:0.7546,model5:0.7537  ->    emsamble]0.8122


In [87]:
sub = df_test.copy()    
sub['ensemble'] = np.where(sub['ensemble'] < 0.5, 0, 1)
sub = sub[['index', 'ensemble']] 
sub.to_csv('sub/submission_rf+lgb+tab.csv', index=None, header=False,)

## スタッキング

In [12]:
x, y = df_train[['pred1', 'pred2','pred3', ]], df_train[['true']]
oof = np.zeros(len(x))
models = []

cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x,y))
for nfold in np.arange(5):
    idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
    x_tr, y_tr = x.loc[idx_tr, :], y.loc[idx_tr, :]
    x_va, y_va = x.loc[idx_va, :], y.loc[idx_tr, :]
    
    model = Lasso(alpha=0.01)
    model.fit(x_tr, y_tr)
    models.append(model)
    
    y_va_pred = model.predict(x_va)
    oof[idx_va] = y_va_pred
    
df_train['stacking'] = oof
df_train['stacking'] = df_train['stacking'].clip(lower=0, upper=1)
df_train.head(20)

,true,pred1,pred2,pred3,pred4,pred5,ensemble,stacking
0,1.0,0.463801,0.628059,0.663979,0.410944,0.402233,0.573132,0.521631
1,0.0,0.083710,0.053503,0.080110,0.109748,0.073798,0.073568,0.074549
2,0.0,0.497738,0.371698,0.792315,0.385019,0.412125,0.548299,0.376201
3,0.0,0.638009,0.586979,0.982137,0.447461,0.602159,0.725938,0.525577
4,1.0,0.800905,0.561965,0.993841,0.492107,0.689416,0.787104,0.560264
5,0.0,0.061086,0.119255,0.287218,0.125439,0.173991,0.146376,0.139218
6,1.0,0.090498,0.051957,0.186049,0.113324,0.124462,0.107601,0.085694
7,1.0,0.518100,0.373684,0.889087,0.463715,0.519163,0.586071,0.398197
8,0.0,0.185520,0.118094,0.284754,0.210352,0.202901,0.195063,0.150160
9,0.0,0.352941,0.288413,0.856990,0.344568,0.347957,0.484797,0.297170


In [13]:
evaluate_ensemble(df_train, col_pred='stacking')

[auc] model1:0.8053,model2:0.8182,model3:0.7669,model4:0.7546,model5:0.7537  ->    emsamble]0.8174


In [16]:
sub = df_test.copy()    
sub['ensemble'] = np.where(sub['ensemble'] < 0.5, 0, 1)
sub = sub[['index', 'ensemble']] 
sub.to_csv('ensamble/sub/submission_stack_w.csv', index=None, header=False,)